In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..
%pwd

/home/markus/Documents/regression-prediction


'/home/markus/Documents/regression-prediction'

In [3]:
from experiments.modeleval_utils import *

/home/markus/Documents/regression-prediction/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
import pickle
with  open('experiments/results/evaluations.pickle', 'rb') as f:
    evaluations_by_config = pickle.load(f)

/home/markus/Documents/regression-prediction/venv/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [5]:
scoring = 'average_precision'
all_sampler_ranks = []
for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance')]: #('bugbug_buglevel', 'regression')
    for feature_type in ['traditional', 'bow']:
        for model in models:
            try:
                results = get_results(data, feature_type, target, scoring, model)
                results = results.replace({
                    None: 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })
                sampler_ranks = results.groupby('param_sampler')[['mean_test_score']].max().sort_values('mean_test_score', ascending=False)
                sampler_ranks['rank'] = np.arange(1,5)
                sampler_ranks['model'] = np.full(4, model)
                all_sampler_ranks.append(sampler_ranks)
            except FileNotFoundError:
                pass
all_sampler_ranks = pd.concat(all_sampler_ranks)
all_sampler_ranks

,mean_test_score,rank,model
param_sampler,,,
No sampling,0.030437,1,lr
Random Over-Sampling,0.029427,2,lr
SMOTE,0.028674,3,lr
Random Under-Sampling,0.026997,4,lr
No sampling,0.036504,1,svm
...,...,...,...
SMOTE,0.113335,4,rf
No sampling,0.135756,1,xgb
Random Under-Sampling,0.110757,2,xgb


In [6]:
t = all_sampler_ranks.groupby('param_sampler')[['rank']].mean().sort_values('rank', ascending=True)
t.index.name = 'Sampling Method'
t = t.rename({'rank': 'Average Rank'}, axis=1)
t

,Average Rank
Sampling Method,
No sampling,1.888889
Random Over-Sampling,2.222222
Random Under-Sampling,2.888889
SMOTE,3.000000


In [7]:
print(t.to_latex())

\begin{tabular}{lr}
\toprule
{} &  Average Rank \\
Sampling Method       &               \\
\midrule
No sampling           &      1.888889 \\
Random Over-Sampling  &      2.222222 \\
Random Under-Sampling &      2.888889 \\
SMOTE                 &      3.000000 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_438/3674067139.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(t.to_latex())


In [8]:
scoring = 'average_precision'
best_sampler = []
for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance'), ('bugbug_buglevel', 'regression')]:
    for feature_type in ['traditional', 'bow']:
        for model in models:
            try:
                results = get_results(data, feature_type, target, scoring, model)
                results = results.replace({
                    None: 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })
                best_sampler.append({
                    'Data': data,
                    'Feature Type': feature_type,
                    'Target': target,
                    'Model': model_names[model],
                    'Sampling':results.iloc[0]['param_sampler'],
                    'AP (val.)': results.iloc[0]['mean_test_score']
                    })
            except FileNotFoundError:
                pass

best_sampler = pd.DataFrame(best_sampler)
best_sampler = best_sampler.replace({
    'fixed_defect_szz': 'SZZ',
    'bugbug_buglevel': 'bugbug'
})
best_sampler = best_sampler.replace({
    'performance': 'perf. regressions',
    'regression': 'all regressions'
})
best_sampler.set_index(pd.MultiIndex.from_frame(best_sampler[['Data', 'Target', 'Feature Type']]), inplace=True)
best_sampler = best_sampler.drop(['Data', 'Target', 'Feature Type'], axis=1)
best_sampler

Model        Sampling  \
Data   Target            Feature Type                                           
SZZ    perf. regressions traditional      Logistic Regression     No sampling   
                         traditional   Support Vector Machine     No sampling   
                         traditional            Random Forest   Over-Sampling   
                         traditional                  XGBoost   Over-Sampling   
                         traditional   Multi-Layer Perceptron     No sampling   
                         bow              Logistic Regression     No sampling   
                         bow           Support Vector Machine   Over-Sampling   
                         bow                    Random Forest     No sampling   
                         bow                          XGBoost     No sampling   
bugbug perf. regressions traditional      Logistic Regression           SMOTE   
                         traditional   Support Vector Machine           SMOTE   
                         traditional            Random Forest   Over-Sampling   
                         traditional                  XGBoost     No sampling   
                         traditional   Multi-Layer Perceptron           SMOTE   
                         bow              Logistic Regression     No sampling   
                         bow           Support Vector Machine  Under-Sampling   
                         bow                    Random Forest  Under-Sampling   
                         bow                          XGBoost     No sampling   
       all regressions   traditional      Logistic Regression   Over-Sampling   
                         traditional   Support Vector Machine   Over-Sampling   
                         traditional            Random Forest     No sampling   
                         traditional                  XGBoost     No sampling   
                         traditional   Multi-Layer Perceptron     No sampling   
                         bow              Logistic Regression     No sampling   
                         bow           Support Vector Machine   Over-Sampling   
                         bow                    Random Forest     No sampling   
                         bow                          XGBoost     No sampling   

                                       AP (val.)  
Data   Target            Feature Type             
SZZ    perf. regressions traditional    0.030437  
                         traditional    0.036504  
                         traditional    0.052774  
                         traditional    0.058965  
                         traditional    0.037452  
                         bow            0.066986  
                         bow            0.056291  
                         bow            0.062738  
                         bow            0.076547  
bugbug perf. regressions traditional    0.076748  
                         traditional    0.080390  
                         traditional    0.105484  
                         traditional    0.108834  
                         traditional    0.086856  
                         bow            0.124864  
                         bow            0.118127  
                         bow            0.134312  
                         bow            0.135756  
       all regressions   traditional    0.372885  
                         traditional    0.383640  
                         traditional    0.423690  
                         traditional    0.423581  
                         traditional    0.387499  
                         bow            0.375307  
                         bow            0.366576  
                         bow            0.415460  
                         bow            0.402749

In [9]:
print(best_sampler.to_latex(index=True))

\begin{tabular}{lllllr}
\toprule
       &                 &     &                   Model &        Sampling &  AP (val.) \\
Data & Target & Feature Type &                         &                 &            \\
\midrule
SZZ & perf. regressions & traditional &     Logistic Regression &     No sampling &   0.030437 \\
       &                 & traditional &  Support Vector Machine &     No sampling &   0.036504 \\
       &                 & traditional &           Random Forest &   Over-Sampling &   0.052774 \\
       &                 & traditional &                 XGBoost &   Over-Sampling &   0.058965 \\
       &                 & traditional &  Multi-Layer Perceptron &     No sampling &   0.037452 \\
       &                 & bow &     Logistic Regression &     No sampling &   0.066986 \\
       &                 & bow &  Support Vector Machine &   Over-Sampling &   0.056291 \\
       &                 & bow &           Random Forest &     No sampling &   0.062738 \\
       &   

/tmp/ipykernel_438/346280886.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(best_sampler.to_latex(index=True))


In [10]:
best_sampler = []

scoring = 'average_precision'
for config, evaluations in evaluations_by_config.items():
    ws = config.split('_')
    feature_type = ws.pop()
    target = ws.pop()
    data = '_'.join(ws)

    amax = np.argmax([evaluation['test']['regression']['f1'] for evaluation in evaluations])
    evaluation = evaluations[amax]

    sampler = None
    if evaluation['model'] == 'tpot':
        sampler = None
    elif evaluation['best_params']:
        sampler = evaluation['best_params']['sampler']

    best_sampler.append({
        'Data': data,
        'Target': target,
        'Feature Type': feature_type,
        'Model':  model_names[evaluation['model']],
        'recall': evaluation['test']['regression']['recall'],
        'precision': evaluation['test']['regression']['precision'],
        'F1': evaluation['test']['regression']['f1'],
        'Sampling Method': str(sampler)
        })

best_sampler = pd.DataFrame(best_sampler)
best_sampler = best_sampler.replace({
                    'None': 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })

best_sampler = best_sampler.replace({
    'fixed_defect_szz': 'SZZ',
    'bugbug_buglevel': 'bugbug'
})
best_sampler = best_sampler.replace({
    'performance': 'perf. regressions',
    'regression': 'all regressions'
})

best_sampler.set_index(pd.MultiIndex.from_frame(best_sampler[['Data', 'Target', 'Feature Type']]), inplace=True)
best_sampler = best_sampler.drop(['Data', 'Target', 'Feature Type'], axis=1)

best_sampler    

Model    recall  \
Data   Target            Feature Type                                     
SZZ    perf. regressions traditional   Multi-Layer Perceptron  0.090909   
                         bow           Support Vector Machine  0.181818   
bugbug perf. regressions traditional      Logistic Regression  0.294606   
                         bow              Logistic Regression  0.153527   
       all regressions   traditional                  XGBoost  0.527504   
                         bow                          XGBoost  0.406206   

                                       precision        F1  \
Data   Target            Feature Type                        
SZZ    perf. regressions traditional    0.017699  0.029630   
                         bow            0.028777  0.049689   
bugbug perf. regressions traditional    0.108397  0.158482   
                         bow            0.202186  0.174528   
       all regressions   traditional    0.377016  0.439741   
                         bow            0.416787  0.411429   

                                            Sampling Method  
Data   Target            Feature Type                        
SZZ    perf. regressions traditional            No sampling  
                         bow           Random Over-Sampling  
bugbug perf. regressions traditional                  SMOTE  
                         bow                    No sampling  
       all regressions   traditional            No sampling  
                         bow                    No sampling

In [11]:
print(best_sampler.to_latex(index=True))

\begin{tabular}{llllrrrl}
\toprule
       &                 &     &                   Model &    recall &  precision &        F1 &       Sampling Method \\
Data & Target & Feature Type &                         &           &            &           &                       \\
\midrule
SZZ & perf. regressions & traditional &  Multi-Layer Perceptron &  0.090909 &   0.017699 &  0.029630 &           No sampling \\
       &                 & bow &  Support Vector Machine &  0.181818 &   0.028777 &  0.049689 &  Random Over-Sampling \\
bugbug & perf. regressions & traditional &     Logistic Regression &  0.294606 &   0.108397 &  0.158482 &                 SMOTE \\
       &                 & bow &     Logistic Regression &  0.153527 &   0.202186 &  0.174528 &           No sampling \\
       & all regressions & traditional &                 XGBoost &  0.527504 &   0.377016 &  0.439741 &           No sampling \\
       &                 & bow &                 XGBoost &  0.406206 &   0.416787 &  0

/tmp/ipykernel_438/346280886.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(best_sampler.to_latex(index=True))
